In [8]:
import json
import re

medal_images = json.loads(open("勋章图片.json").read())
medal_levels = json.loads(open("勋章内容映射.json").read())
medal_library = json.loads(open("勋章博物馆.json").read())

# 属性价值权重
收益权重映射 = {
    '金币': 1,
    '血液': 1,
    '旅程': 30,
    '咒术': 5,
    '知识': 50,
    '灵魂': 1000,
    '堕落': 0  # 堕落不计入
}

def calculate_attribute_value(attributes_dict, weight_mapping):
    """计算属性字典的总价值"""
    total = 0
    for attr, value in attributes_dict.items():
        if attr in weight_mapping and attr != '堕落':
            total += value * weight_mapping[attr]
    return total


def parse_medal_data(medal_levels, medal_attributes):
    """
    解析勋章数据并计算收益和性价比
    
    Args:
        medal_levels: 勋章原始数据字典
        medal_attributes: 属性列表
    
    Returns:
        解析后的勋章数据字典（包含收益和性价比）
    """
    result = {}
    
    for medal_name, medal_info in medal_levels.items():
        lines = medal_info.strip().split('\n')
        
        parsed_medal = {
            '勋章名称': medal_name,
            '勋章类型': '',
            '入手条件': '',
            '商店售价': {},
            '等级': [],
            '发帖最高收益等级': None,
            '回帖最高收益等级': None,
            '发帖性价比': 0,
            '回帖性价比': 0,
            '排行': {
                '发帖收益': {'总排行': 0, '同类型排行': 0},
                '回帖收益': {'总排行': 0, '同类型排行': 0},
                '发帖性价比': {'总排行': 0, '同类型排行': 0},
                '回帖性价比': {'总排行': 0, '同类型排行': 0}
            }
        }
        
        for line in lines:
            line = line.strip()
            
            # 提取勋章类型
            if '【勋章类型】' in line:
                parsed_medal['勋章类型'] = line.replace('【勋章类型】', '')
            
            # 提取入手条件
            elif '【入手条件】' in line:
                parsed_medal['入手条件'] = line.replace('【入手条件】', '')
            
            # 提取商店售价
            elif '【商店售价】' in line:
                price_text = line.replace('【商店售价】', '')
                price_match = re.search(r'(\d+)(\D+)', price_text)
                if price_match:
                    value = int(price_match.group(1))
                    attr = price_match.group(2).strip()
                    parsed_medal['商店售价'] = {attr: value}
            
            # 提取等级信息
            elif '【等级' in line or '【 Max 】' in line:
                level_data = parse_level_line(medal_name, line, medal_attributes)
                parsed_medal['等级'].append(level_data)
        
        # 计算每个等级的收益和累计成本
        calculate_level_benefits(parsed_medal)
        
        # 计算勋章整体的最高收益等级和性价比
        calculate_medal_statistics(parsed_medal)
        
        result[medal_name] = parsed_medal
    
    # 计算排行榜
    calculate_rankings(result)
    
    return result


def parse_level_line(medal_name, line, medal_attributes):
    """解析单个等级行"""
    level_info = {
        '等级': '',
        '概率': 0,
        '发帖属性': {},
        '回帖属性': {},
        '升级消耗': {},
        '图片': {},
        '博物馆': '',
        '升级条件': ''
    }
    
    # 提取等级名称
    if '【 Max 】' in line:
        level_info['等级'] = 'Max'
        content = line.split('【 Max 】')[1]
    else:
        level_match = re.search(r'【等级(\d+)】', line)
        if level_match:
            level_info['等级'] = int(level_match.group(1))
            content = line.split('】', 1)[1]
        else:
            return level_info
    
    # 分割升级消耗
    parts = content.split('▕▏')
    effects_part = parts[0].strip()
    upgrade_part = parts[1].strip() if len(parts) > 1 else ''
    
    # 提取概率
    prob_match = re.search(r'(\d+)%', effects_part)
    if prob_match:
        level_info['概率'] = int(prob_match.group(1))
    
    # 提取属性效果（支持正负值）
    for attr in medal_attributes:
        # 发帖属性：匹配 +数字 或 -数字
        post_pattern = rf'发帖[^、▕▏]*?{attr}([+\-]\d+)'
        post_matches = re.findall(post_pattern, effects_part)
        if post_matches:
            total = sum(int(m) for m in post_matches)
            if total != 0:
                level_info['发帖属性'][attr] = total
        
        # 回帖属性：匹配 +数字 或 -数字
        reply_pattern = rf'回帖[^、▕▏]*?{attr}([+\-]\d+)'
        reply_matches = re.findall(reply_pattern, effects_part)
        if reply_matches:
            total = sum(int(m) for m in reply_matches)
            if total != 0:
                level_info['回帖属性'][attr] = total
    
    # 提取升级消耗
    if upgrade_part and '升级条件' in upgrade_part:
        level_info['升级条件'] = upgrade_part
        upgrade_text = upgrade_part.replace('升级条件：', '').replace('消耗', '')
        for attr in medal_attributes:
            upgrade_pattern = rf'(\d+){attr}'
            upgrade_match = re.search(upgrade_pattern, upgrade_text)
            if upgrade_match:
                level_info['升级消耗'][attr] = int(upgrade_match.group(1))

    # 提取图片
    img_info = medal_images.get(medal_name)
    if img_info:
        img_level_info = img_info.get(str(level_info['等级']))
        if img_level_info:
            level_info['图片']['url'] = img_level_info[0]
            level_info['图片']['size'] = img_level_info[1]

    # 提取博物馆地址
    level_info['博物馆'] = medal_library.get(medal_name) or ''
    
    return level_info


def calculate_level_benefits(parsed_medal):
    """计算每个等级的收益和累计成本（考虑概率）"""
    # 商店购买成本
    is_xxx = parsed_medal['勋章名称'] == '双生蛋'
    shop_cost = calculate_attribute_value(parsed_medal.get('商店售价', {}), 收益权重映射)
    
    cumulative_cost = shop_cost  # 累计总消耗

    if is_xxx:
        print(cumulative_cost)
    
    for i, level_data in enumerate(parsed_medal['等级']):
        # 获取触发概率
        probability = level_data.get('概率', 0) / 100.0
        
        # 1. 计算发帖基础收益
        post_base_benefit = calculate_attribute_value(level_data.get('发帖属性', {}), 收益权重映射)
        
        # 2. 计算回帖基础收益
        reply_base_benefit = calculate_attribute_value(level_data.get('回帖属性', {}), 收益权重映射)
        
        # 3. 考虑概率后的期望收益
        post_benefit = post_base_benefit * probability
        reply_benefit = reply_base_benefit * probability
        
        # 4. 计算本级升级成本
        upgrade_cost = calculate_attribute_value(level_data.get('升级消耗', {}), 收益权重映射)
        if is_xxx:
            print('upgrade_cost', upgrade_cost)
        
        # 5. 累计总消耗
        cumulative_cost += upgrade_cost
        
        # 6. 存储到等级数据中
        level_data['发帖基础收益'] = post_base_benefit
        level_data['回帖基础收益'] = reply_base_benefit
        level_data['发帖期望收益'] = round(post_benefit, 2)
        level_data['回帖期望收益'] = round(reply_benefit, 2)
        level_data['升级成本'] = upgrade_cost
        # level_data['累计总消耗'] = cumulative_cost

    parsed_medal['累计总消耗'] = cumulative_cost


def calculate_medal_statistics(parsed_medal):
    """计算勋章的最高收益等级和性价比"""
    max_post_benefit = 0
    max_reply_benefit = 0
    max_post_level = None
    max_reply_level = None
    final_cost = 0
    
    for level_data in parsed_medal['等级']:
        # 使用期望收益（已考虑概率）
        post_benefit = level_data.get('发帖期望收益', 0)
        reply_benefit = level_data.get('回帖期望收益', 0)
        
        # 找到最高期望收益等级
        if post_benefit > max_post_benefit:
            max_post_benefit = post_benefit
            max_post_level = level_data['等级']
        
        if reply_benefit > max_reply_benefit:
            max_reply_benefit = reply_benefit
            max_reply_level = level_data['等级']
        
        # 获取最终累计消耗
        final_cost = level_data.get('累计总消耗', 0)
    
    # 存储最高收益等级
    parsed_medal['发帖最高收益等级'] = {
        '等级': max_post_level,
        '期望收益': round(max_post_benefit, 2)
    }
    parsed_medal['回帖最高收益等级'] = {
        '等级': max_reply_level,
        '期望收益': round(max_reply_benefit, 2)
    }
    
    # 计算性价比（使用满级的累计总消耗）
    if final_cost > 0:
        parsed_medal['发帖性价比'] = round(max_post_benefit / final_cost, 6)
        parsed_medal['回帖性价比'] = round(max_reply_benefit / final_cost, 6)
    else:
        parsed_medal['发帖性价比'] = max_post_benefit if max_post_benefit > 0 else 0
        parsed_medal['回帖性价比'] = max_reply_benefit if max_reply_benefit > 0 else 0


def calculate_rankings(parsed_data):
    """计算各维度的排行榜（总排行和同类型排行）"""
    # 按类型分组
    type_groups = {}
    all_medals = []
    
    for medal_name, medal_info in parsed_data.items():
        medal_type = medal_info.get('勋章类型', '未分类')
        
        medal_stats = {
            'name': medal_name,
            'type': medal_type,
            'post_benefit': medal_info['发帖最高收益等级'].get('期望收益', 0),
            'reply_benefit': medal_info['回帖最高收益等级'].get('期望收益', 0),
            'post_ratio': medal_info.get('发帖性价比', 0),
            'reply_ratio': medal_info.get('回帖性价比', 0)
        }
        
        all_medals.append(medal_stats)
        
        if medal_type not in type_groups:
            type_groups[medal_type] = []
        type_groups[medal_type].append(medal_stats)
    
    # 1. 计算总排行（4个维度）
    # 发帖收益总排行
    all_sorted_post_benefit = sorted(all_medals, key=lambda x: x['post_benefit'], reverse=True)
    for i, medal_stats in enumerate(all_sorted_post_benefit, 1):
        parsed_data[medal_stats['name']]['排行']['发帖收益']['总排行'] = i
    
    # 回帖收益总排行
    all_sorted_reply_benefit = sorted(all_medals, key=lambda x: x['reply_benefit'], reverse=True)
    for i, medal_stats in enumerate(all_sorted_reply_benefit, 1):
        parsed_data[medal_stats['name']]['排行']['回帖收益']['总排行'] = i
    
    # 发帖性价比总排行
    all_sorted_post_ratio = sorted(all_medals, key=lambda x: x['post_ratio'], reverse=True)
    for i, medal_stats in enumerate(all_sorted_post_ratio, 1):
        parsed_data[medal_stats['name']]['排行']['发帖性价比']['总排行'] = i
    
    # 回帖性价比总排行
    all_sorted_reply_ratio = sorted(all_medals, key=lambda x: x['reply_ratio'], reverse=True)
    for i, medal_stats in enumerate(all_sorted_reply_ratio, 1):
        parsed_data[medal_stats['name']]['排行']['回帖性价比']['总排行'] = i
    
    # 2. 计算同类型排行（4个维度）
    for medal_type, medals_in_type in type_groups.items():
        # 发帖收益同类型排行
        type_sorted_post_benefit = sorted(medals_in_type, key=lambda x: x['post_benefit'], reverse=True)
        for i, medal_stats in enumerate(type_sorted_post_benefit, 1):
            parsed_data[medal_stats['name']]['排行']['发帖收益']['同类型排行'] = i
        
        # 回帖收益同类型排行
        type_sorted_reply_benefit = sorted(medals_in_type, key=lambda x: x['reply_benefit'], reverse=True)
        for i, medal_stats in enumerate(type_sorted_reply_benefit, 1):
            parsed_data[medal_stats['name']]['排行']['回帖收益']['同类型排行'] = i
        
        # 发帖性价比同类型排行
        type_sorted_post_ratio = sorted(medals_in_type, key=lambda x: x['post_ratio'], reverse=True)
        for i, medal_stats in enumerate(type_sorted_post_ratio, 1):
            parsed_data[medal_stats['name']]['排行']['发帖性价比']['同类型排行'] = i
        
        # 回帖性价比同类型排行
        type_sorted_reply_ratio = sorted(medals_in_type, key=lambda x: x['reply_ratio'], reverse=True)
        for i, medal_stats in enumerate(type_sorted_reply_ratio, 1):
            parsed_data[medal_stats['name']]['排行']['回帖性价比']['同类型排行'] = i


# 使用示例
medal_attributes = ['金币', '血液', '旅程', '咒术', '知识', '灵魂', '堕落']
parsed_data = parse_medal_data(medal_levels, medal_attributes)

# 导出结果
with open('勋章完整分析.json', 'w', encoding='utf-8') as f:
    json.dump(parsed_data, f, ensure_ascii=False, indent=2)

print("勋章数据解析完成，已保存到 勋章完整分析.json")


# 辅助函数：打印排行榜
def print_top_medals(parsed_data, rank_type='发帖性价比', category=None, top_n=10):
    """
    打印指定维度的排行榜
    
    Args:
        parsed_data: 解析后的勋章数据
        rank_type: 排行类型 ('发帖收益', '回帖收益', '发帖性价比', '回帖性价比')
        category: 勋章类型筛选，None表示全部
        top_n: 显示前N名
    """
    medals_list = []
    
    for medal_name, medal_info in parsed_data.items():
        # 类型筛选
        if category and medal_info.get('勋章类型', '') != category:
            continue
        
        medals_list.append({
            '勋章名称': medal_name,
            '勋章类型': medal_info.get('勋章类型', ''),
            '发帖性价比': medal_info.get('发帖性价比', 0),
            '回帖性价比': medal_info.get('回帖性价比', 0),
            '发帖收益': medal_info.get('发帖最高收益等级', {}).get('期望收益', 0),
            '回帖收益': medal_info.get('回帖最高收益等级', {}).get('期望收益', 0),
            '发帖最高等级': medal_info.get('发帖最高收益等级', {}).get('等级', '-'),
            '回帖最高等级': medal_info.get('回帖最高收益等级', {}).get('等级', '-'),
            '排行': medal_info.get('排行', {})
        })
    
    # 根据不同标准排序
    if rank_type == '发帖性价比':
        sorted_medals = sorted(medals_list, key=lambda x: x['发帖性价比'], reverse=True)
    elif rank_type == '回帖性价比':
        sorted_medals = sorted(medals_list, key=lambda x: x['回帖性价比'], reverse=True)
    elif rank_type == '发帖收益':
        sorted_medals = sorted(medals_list, key=lambda x: x['发帖收益'], reverse=True)
    elif rank_type == '回帖收益':
        sorted_medals = sorted(medals_list, key=lambda x: x['回帖收益'], reverse=True)
    else:
        sorted_medals = medals_list
    
    # 打印结果
    category_text = f"【{category}】" if category else "【全部勋章】"
    print(f"\n{'='*90}")
    print(f"{category_text} {rank_type} 排行榜 Top {top_n}")
    print(f"{'='*90}")
    
    for i, medal in enumerate(sorted_medals[:top_n], 1):
        rank_info = medal['排行']
        print(f"\n{i}. {medal['勋章名称']} ({medal['勋章类型']})")
        print(f"   ┌─ 发帖数据:")
        print(f"   │  期望收益: {medal['发帖收益']:.2f} | 性价比: {medal['发帖性价比']:.6f} | 最高等级: {medal['发帖最高等级']}")
        print(f"   │  收益排行 → 总排行: 第{rank_info.get('发帖收益', {}).get('总排行', '-')}名 | 同类型: 第{rank_info.get('发帖收益', {}).get('同类型排行', '-')}名")
        print(f"   │  性价比排行 → 总排行: 第{rank_info.get('发帖性价比', {}).get('总排行', '-')}名 | 同类型: 第{rank_info.get('发帖性价比', {}).get('同类型排行', '-')}名")
        print(f"   └─ 回帖数据:")
        print(f"      期望收益: {medal['回帖收益']:.2f} | 性价比: {medal['回帖性价比']:.6f} | 最高等级: {medal['回帖最高等级']}")
        print(f"      收益排行 → 总排行: 第{rank_info.get('回帖收益', {}).get('总排行', '-')}名 | 同类型: 第{rank_info.get('回帖收益', {}).get('同类型排行', '-')}名")
        print(f"      性价比排行 → 总排行: 第{rank_info.get('回帖性价比', {}).get('总排行', '-')}名 | 同类型: 第{rank_info.get('回帖性价比', {}).get('同类型排行', '-')}名")


def print_category_summary(parsed_data):
    """打印各类别的勋章统计"""
    type_stats = {}
    
    for medal_name, medal_info in parsed_data.items():
        medal_type = medal_info.get('勋章类型', '未分类')
        
        if medal_type not in type_stats:
            type_stats[medal_type] = {
                'count': 0,
                'avg_post_benefit': 0,
                'avg_reply_benefit': 0,
                'avg_post_ratio': 0,
                'avg_reply_ratio': 0,
                'total_post_benefit': 0,
                'total_reply_benefit': 0,
                'total_post_ratio': 0,
                'total_reply_ratio': 0
            }
        
        type_stats[medal_type]['count'] += 1
        type_stats[medal_type]['total_post_benefit'] += medal_info.get('发帖最高收益等级', {}).get('期望收益', 0)
        type_stats[medal_type]['total_reply_benefit'] += medal_info.get('回帖最高收益等级', {}).get('期望收益', 0)
        type_stats[medal_type]['total_post_ratio'] += medal_info.get('发帖性价比', 0)
        type_stats[medal_type]['total_reply_ratio'] += medal_info.get('回帖性价比', 0)
    
    # 计算平均值
    for medal_type, stats in type_stats.items():
        if stats['count'] > 0:
            stats['avg_post_benefit'] = stats['total_post_benefit'] / stats['count']
            stats['avg_reply_benefit'] = stats['total_reply_benefit'] / stats['count']
            stats['avg_post_ratio'] = stats['total_post_ratio'] / stats['count']
            stats['avg_reply_ratio'] = stats['total_reply_ratio'] / stats['count']
    
    # print(f"\n{'='*90}")
    # print("勋章类别统计摘要")
    # print(f"{'='*90}")
    
    # for medal_type, stats in sorted(type_stats.items(), key=lambda x: x[1]['avg_post_ratio'], reverse=True):
    #     print(f"\n【{medal_type}】")
    #     print(f"   勋章数量: {stats['count']}")
    #     print(f"   平均发帖收益: {stats['avg_post_benefit']:.2f} | 平均发帖性价比: {stats['avg_post_ratio']:.6f}")
    #     print(f"   平均回帖收益: {stats['avg_reply_benefit']:.2f} | 平均回帖性价比: {stats['avg_reply_ratio']:.6f}")


def export_ranking_report(parsed_data, output_file='勋章排行报告.txt'):
    """导出详细的排行报告"""
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("="*90 + "\n")
        f.write("勋章完整排行报告\n")
        f.write("="*90 + "\n\n")
        
        # 获取所有类型
        all_types = set()
        for medal_info in parsed_data.values():
            all_types.add(medal_info.get('勋章类型', '未分类'))
        
        # 四个维度的全部排行
        dimensions = [
            ('发帖收益', lambda x: x[1].get('发帖最高收益等级', {}).get('期望收益', 0)),
            ('回帖收益', lambda x: x[1].get('回帖最高收益等级', {}).get('期望收益', 0)),
            ('发帖性价比', lambda x: x[1].get('发帖性价比', 0)),
            ('回帖性价比', lambda x: x[1].get('回帖性价比', 0))
        ]
        
        all_medals = [(name, info) for name, info in parsed_data.items()]
        
        for dim_name, sort_key in dimensions:
            f.write("\n" + "="*90 + "\n")
            f.write(f"【全部勋章】{dim_name} Top 20\n")
            f.write("="*90 + "\n")
            
            sorted_medals = sorted(all_medals, key=sort_key, reverse=True)
            
            for i, (name, info) in enumerate(sorted_medals[:20], 1):
                rank_info = info.get('排行', {})
                f.write(f"\n{i}. {name} ({info.get('勋章类型', '')})\n")
                
                if '收益' in dim_name:
                    if '发帖' in dim_name:
                        value = info.get('发帖最高收益等级', {}).get('期望收益', 0)
                        f.write(f"   期望收益: {value:.2f}\n")
                        f.write(f"   总排行: 第{rank_info.get('发帖收益', {}).get('总排行', '-')}名 | ")
                        f.write(f"同类型排行: 第{rank_info.get('发帖收益', {}).get('同类型排行', '-')}名\n")
                    else:
                        value = info.get('回帖最高收益等级', {}).get('期望收益', 0)
                        f.write(f"   期望收益: {value:.2f}\n")
                        f.write(f"   总排行: 第{rank_info.get('回帖收益', {}).get('总排行', '-')}名 | ")
                        f.write(f"同类型排行: 第{rank_info.get('回帖收益', {}).get('同类型排行', '-')}名\n")
                else:
                    if '发帖' in dim_name:
                        value = info.get('发帖性价比', 0)
                        f.write(f"   性价比: {value:.6f}\n")
                        f.write(f"   总排行: 第{rank_info.get('发帖性价比', {}).get('总排行', '-')}名 | ")
                        f.write(f"同类型排行: 第{rank_info.get('发帖性价比', {}).get('同类型排行', '-')}名\n")
                    else:
                        value = info.get('回帖性价比', 0)
                        f.write(f"   性价比: {value:.6f}\n")
                        f.write(f"   总排行: 第{rank_info.get('回帖性价比', {}).get('总排行', '-')}名 | ")
                        f.write(f"同类型排行: 第{rank_info.get('回帖性价比', {}).get('同类型排行', '-')}名\n")
        
        # 各类别排行
        for medal_type in sorted(all_types):
            f.write("\n\n" + "="*90 + "\n")
            f.write(f"【{medal_type}】类别详细排行\n")
            f.write("="*90 + "\n")
            
            type_medals = [(name, info) for name, info in parsed_data.items() 
                          if info.get('勋章类型', '') == medal_type]
            
            for dim_name, sort_key in dimensions:
                f.write(f"\n--- {dim_name} ---\n")
                sorted_type = sorted(type_medals, key=sort_key, reverse=True)
                
                for i, (name, info) in enumerate(sorted_type[:10], 1):
                    if '发帖收益' in dim_name:
                        value = info.get('发帖最高收益等级', {}).get('期望收益', 0)
                        f.write(f"{i}. {name} - 收益: {value:.2f}\n")
                    elif '回帖收益' in dim_name:
                        value = info.get('回帖最高收益等级', {}).get('期望收益', 0)
                        f.write(f"{i}. {name} - 收益: {value:.2f}\n")
                    elif '发帖性价比' in dim_name:
                        value = info.get('发帖性价比', 0)
                        f.write(f"{i}. {name} - 性价比: {value:.6f}\n")
                    else:
                        value = info.get('回帖性价比', 0)
                        f.write(f"{i}. {name} - 性价比: {value:.6f}\n")
    
    print(f"\n排行报告已导出到: {output_file}")


def get_medal_details(parsed_data, medal_name):
    """查看单个勋章的详细信息"""
    if medal_name not in parsed_data:
        print(f"未找到勋章: {medal_name}")
        return
    
    medal = parsed_data[medal_name]
    
    # print(f"\n{'='*90}")
    # print(f"勋章详情: {medal_name}")
    # print(f"{'='*90}")
    # print(f"勋章类型: {medal.get('勋章类型', '未知')}")
    # print(f"入手条件: {medal.get('入手条件', '未知')}")
    # print(f"商店售价: {medal.get('商店售价', {})}")
    
    # print(f"\n--- 收益总览 ---")
    # print(f"发帖最高收益: {medal.get('发帖最高收益等级', {}).get('期望收益', 0):.2f} (等级 {medal.get('发帖最高收益等级', {}).get('等级', '-')})")
    # print(f"回帖最高收益: {medal.get('回帖最高收益等级', {}).get('期望收益', 0):.2f} (等级 {medal.get('回帖最高收益等级', {}).get('等级', '-')})")
    # print(f"发帖性价比: {medal.get('发帖性价比', 0):.6f}")
    # print(f"回帖性价比: {medal.get('回帖性价比', 0):.6f}")
    
    # print(f"\n--- 排行信息 ---")
    # rank_info = medal.get('排行', {})
    # print(f"发帖收益排行: 总排行第 {rank_info.get('发帖收益', {}).get('总排行', '-')} 名 | 同类型第 {rank_info.get('发帖收益', {}).get('同类型排行', '-')} 名")
    # print(f"回帖收益排行: 总排行第 {rank_info.get('回帖收益', {}).get('总排行', '-')} 名 | 同类型第 {rank_info.get('回帖收益', {}).get('同类型排行', '-')} 名")
    # print(f"发帖性价比排行: 总排行第 {rank_info.get('发帖性价比', {}).get('总排行', '-')} 名 | 同类型第 {rank_info.get('发帖性价比', {}).get('同类型排行', '-')} 名")
    # print(f"回帖性价比排行: 总排行第 {rank_info.get('回帖性价比', {}).get('总排行', '-')} 名 | 同类型第 {rank_info.get('回帖性价比', {}).get('同类型排行', '-')} 名")
    
    # print(f"\n--- 等级详情 ---")
    # for level_data in medal.get('等级', []):
    #     print(f"\n等级 {level_data.get('等级', '-')}:")
    #     print(f"  触发概率: {level_data.get('概率', 0)}%")
    #     print(f"  发帖属性: {level_data.get('发帖属性', {})}")
    #     print(f"  回帖属性: {level_data.get('回帖属性', {})}")
    #     print(f"  发帖基础收益: {level_data.get('发帖基础收益', 0)} | 期望收益: {level_data.get('发帖期望收益', 0):.2f}")
    #     print(f"  回帖基础收益: {level_data.get('回帖基础收益', 0)} | 期望收益: {level_data.get('回帖期望收益', 0):.2f}")
    #     print(f"  升级消耗: {level_data.get('升级消耗', {})}")
    #     print(f"  升级成本: {level_data.get('升级成本', 0)}")
    #     print(f"  累计总消耗: {level_data.get('累计总消耗', 0)}")
    #     if level_data.get('图片', {}).get('url'):
    #         print(f"  图片: {level_data.get('图片', {}).get('url', '')}")
    #     if level_data.get('博物馆'):
    #         print(f"  博物馆: {level_data.get('博物馆', '')}")


def compare_medals(parsed_data, medal_names):
    """对比多个勋章"""
    print(f"\n{'='*90}")
    print(f"勋章对比")
    print(f"{'='*90}\n")
    
    valid_medals = []
    for name in medal_names:
        if name in parsed_data:
            valid_medals.append(name)
        else:
            print(f"警告: 未找到勋章 '{name}'")
    
    if len(valid_medals) < 2:
        print("至少需要2个有效勋章进行对比")
        return
    
    print(f"{'指标':<20}", end='')
    for name in valid_medals:
        print(f"{name:<25}", end='')
    print()
    print("-" * 90)
    
    # 对比基本信息
    print(f"{'勋章类型':<20}", end='')
    for name in valid_medals:
        print(f"{parsed_data[name].get('勋章类型', ''):<25}", end='')
    print()
    
    # 对比收益
    print(f"{'发帖期望收益':<20}", end='')
    for name in valid_medals:
        value = parsed_data[name].get('发帖最高收益等级', {}).get('期望收益', 0)
        print(f"{value:<25.2f}", end='')
    print()
    
    print(f"{'回帖期望收益':<20}", end='')
    for name in valid_medals:
        value = parsed_data[name].get('回帖最高收益等级', {}).get('期望收益', 0)
        print(f"{value:<25.2f}", end='')
    print()
    
    # 对比性价比
    print(f"{'发帖性价比':<20}", end='')
    for name in valid_medals:
        value = parsed_data[name].get('发帖性价比', 0)
        print(f"{value:<25.6f}", end='')
    print()
    
    print(f"{'回帖性价比':<20}", end='')
    for name in valid_medals:
        value = parsed_data[name].get('回帖性价比', 0)
        print(f"{value:<25.6f}", end='')
    print()
    
    # 对比排行
    print("\n排行对比:")
    dimensions = ['发帖收益', '回帖收益', '发帖性价比', '回帖性价比']
    for dim in dimensions:
        print(f"\n{dim}:")
        print(f"  {'总排行':<18}", end='')
        for name in valid_medals:
            rank = parsed_data[name].get('排行', {}).get(dim, {}).get('总排行', '-')
            print(f"第{rank}名{'':<20}", end='')
        print()
        
        print(f"  {'同类型排行':<18}", end='')
        for name in valid_medals:
            rank = parsed_data[name].get('排行', {}).get(dim, {}).get('同类型排行', '-')
            print(f"第{rank}名{'':<20}", end='')
        print()


# # 执行各种分析
# print("\n" + "="*90)
# print("开始勋章数据分析...")
# print("="*90)

# # 1. 打印四个维度的总排行
# print("\n" + "="*90)
# print("【一】四大维度总排行榜")
# print("="*90)
# print_top_medals(parsed_data, '发帖收益', category=None, top_n=10)
# print_top_medals(parsed_data, '回帖收益', category=None, top_n=10)
# print_top_medals(parsed_data, '发帖性价比', category=None, top_n=10)
# print_top_medals(parsed_data, '回帖性价比', category=None, top_n=10)

# # 2. 打印类别统计
# print("\n" + "="*90)
# print("【二】类别统计摘要")
# print("="*90)
# print_category_summary(parsed_data)

# # 3. 示例：查看某个类别的排行
# # 获取所有类型
# all_types = set()
# for medal_info in parsed_data.values():
#     medal_type = medal_info.get('勋章类型', '')
#     if medal_type:
#         all_types.add(medal_type)

# if all_types:
#     sample_type = list(all_types)[0]
#     print("\n" + "="*90)
#     print(f"【三】示例：{sample_type} 类别排行")
#     print("="*90)
#     print_top_medals(parsed_data, '发帖性价比', category=sample_type, top_n=5)

# # 4. 导出详细报告
# print("\n" + "="*90)
# print("【四】导出详细报告")
# print("="*90)
# export_ranking_report(parsed_data)

# # 5. 示例：查看单个勋章详情（取第一个勋章作为示例）
# if parsed_data:
#     sample_medal = list(parsed_data.keys())[0]
#     print("\n" + "="*90)
#     print(f"【五】示例：勋章详情查询")
#     print("="*90)
#     get_medal_details(parsed_data, sample_medal)

# # 6. 示例：对比勋章（取前三个勋章作为示例）
# if len(parsed_data) >= 3:
#     sample_medals = list(parsed_data.keys())[:3]
#     print("\n" + "="*90)
#     print("【六】示例：勋章对比")
#     print("="*90)
#     compare_medals(parsed_data, sample_medals)

# print("\n" + "="*90)
# print("所有分析完成！")
# print("="*90)
# print("\n生成的文件:")
# print("  1. 勋章完整分析.json - 包含所有勋章数据和排行信息")
# print("  2. 勋章排行报告.txt - 详细的文本格式排行报告")
# print("\n可用的查询函数:")
# print("  - get_medal_details(parsed_data, '勋章名称') - 查看单个勋章详情")
# print("  - compare_medals(parsed_data, ['勋章1', '勋章2', ...]) - 对比多个勋章")
# print("  - print_top_medals(parsed_data, '维度', category='类型', top_n=10) - 查看排行榜")
# print("    维度可选: '发帖收益', '回帖收益', '发帖性价比', '回帖性价比'")



377
upgrade_cost 377
upgrade_cost 0
upgrade_cost 0
勋章数据解析完成，已保存到 勋章完整分析.json


In [9]:
parsed_data.get('双生蛋')

{'勋章名称': '双生蛋',
 '勋章类型': '宠物',
 '入手条件': '堕落 >= 177',
 '商店售价': {'金币': 377},
 '等级': [{'等级': 1,
   '概率': 3,
   '发帖属性': {},
   '回帖属性': {'堕落': 1},
   '升级消耗': {'血液': 377},
   '图片': {'url': 'https://img.gamemale.com/album/202510/01/144224dklyukkhl51k0f5o.gif',
    'size': 40},
   '博物馆': '171614',
   '升级条件': '升级条件：消耗377血液',
   '发帖基础收益': 0,
   '回帖基础收益': 0,
   '发帖期望收益': 0.0,
   '回帖期望收益': 0.0,
   '升级成本': 377},
  {'等级': 2,
   '概率': 15,
   '发帖属性': {},
   '回帖属性': {'金币': 3, '堕落': 1},
   '升级消耗': {},
   '图片': {'url': 'https://img.gamemale.com/album/202510/01/144230u7f7fqo1zmvoob3h.gif',
    'size': 40},
   '博物馆': '171614',
   '升级条件': '升级条件：堕落≥17',
   '发帖基础收益': 0,
   '回帖基础收益': 3,
   '发帖期望收益': 0.0,
   '回帖期望收益': 0.45,
   '升级成本': 0},
  {'等级': 'Max',
   '概率': 10,
   '发帖属性': {},
   '回帖属性': {'金币': 1, '堕落': -1},
   '升级消耗': {},
   '图片': {'url': 'https://img.gamemale.com/album/202510/01/144239ui7kzhd0ywt7voja.gif',
    'size': 40},
   '博物馆': '171614',
   '升级条件': '',
   '发帖基础收益': 0,
   '回帖基础收益': 1,
   '发帖期望收益': 0